In [1]:
import pandas as pd
import numpy as np
import os
import logging
from dotenv import load_dotenv
from sqlalchemy import create_engine
import yfinance as yf
import datetime as datetime
from scipy.stats import norm

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

load_dotenv()
# Load environment variables

user = os.getenv('PG_USER')
password = os.getenv('PG_PASSWORD')
host = os.getenv('PG_HOST')
port = os.getenv('PG_PORT')
database = os.getenv('PG_DB')

if not all([user, password, host, port, database]):
    raise ValueError("One or more environment variables are not set. Please check your .env file.")

In [3]:
# FETCH 30 DAY LIVE DATA FROM YAHOO FINANCE
def fetch_recent_data(tickers, period="30d", interval="1d"):
    """
    Fetch recent historical price data from yfinance.
    """
    try:
        logger.info(f"Fetching {period} of data for tickers: {', '.join(tickers)}")
        data = yf.download(tickers, period=period, interval=interval, group_by='ticker', auto_adjust=True)

        if data.empty:
            logger.warning("No data returned from yfinance.")
            return pd.DataFrame()

        if isinstance(data.columns, pd.MultiIndex):
            try:
                return data['Close']
            except KeyError:
                close_data = pd.DataFrame({
                    ticker: data[ticker]['Close']
                    for ticker in data.columns.levels[0]
                    if 'Close' in data[ticker]
                })
                return close_data
        else:
            return data

    except Exception as e:
        logger.error(f"Error fetching stock data: {e}")
        return pd.DataFrame()


In [5]:
# CALCULATE SHARPE RATIO AND PARAMETRIC VAR

def calculate_rolling_risk_metrics(price_df, alpha=0.05, risk_free_rate=0.03):
    """
    Calculate Sharpe Ratio and Parametric VaR for each asset using 30-day data.

    Parameters:
        price_df (pd.DataFrame): Price data (Date index, tickers as columns)
        alpha (float): Confidence level (e.g., 0.05 for 95%)
        risk_free_rate (float): Annual risk-free rate

    Returns:
        pd.DataFrame: One-row DataFrame of metrics per ticker with timestamp.
    """
    if price_df.empty:
        return pd.DataFrame()

    # Calculate daily returns
    returns = price_df.pct_change().dropna()

    # Risk-free daily rate
    rf_daily = (1 + risk_free_rate) ** (1/252) - 1

    # Metrics
    mu = returns.mean()
    sigma = returns.std()
    sharpe_ratios = (mu - rf_daily) / sigma

    z = norm.ppf(1 - alpha)
    var_param = -(mu + z * sigma)

    # Timestamp for export
    timestamp = pd.Timestamp.now()

    # Assemble result
    result_df = pd.DataFrame({
        'ticker': mu.index,
        'sharpe_ratio': sharpe_ratios.values,
        'var_param_95': var_param.values,
        'timestamp': timestamp
    })

    return result_df

# Test Functions

# Example tickers
tickers = ['AAPL', 'MSFT', 'TSLA', 'JNJ', 'NVDA']

# Fetch recent data
price_data = fetch_recent_data(tickers)

# Calculate metrics
rolling_metrics_df = calculate_rolling_risk_metrics(price_data)

# Preview the result
if rolling_metrics_df.empty:
    logger.warning("No metrics calculated. Exiting.")
    exit()
else:
    print("\nSample of calculated rolling risk metrics:")
    print(rolling_metrics_df)

# Simulate past 7 days for testing purposes
from datetime import timedelta

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

try:
    for i in range(7):
        simulated_df = rolling_metrics_df.copy()
        simulated_df['timestamp'] = pd.Timestamp.now() - timedelta(days=i)

        simulated_df.to_sql("rolling_risk_metrics", engine, if_exists='append', index=False)
        logger.info(f"Simulated data for {simulated_df['timestamp'].iloc[0].date()} exported.")
except Exception as e:
    logger.error(f"Error during simulation export: {e}")
finally:
    engine.dispose()
    logger.info("PostgreSQL connection closed after simulation.")

2025-04-16 09:16:09,745 - INFO - Fetching 30d of data for tickers: AAPL, MSFT, TSLA, JNJ, NVDA
[*********************100%***********************]  5 of 5 completed
C:\Users\roman\AppData\Local\Temp\ipykernel_10068\3666117503.py:19: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = price_df.pct_change().dropna()
2025-04-16 09:16:10,142 - INFO - Simulated data for 2025-04-16 exported.
2025-04-16 09:16:10,152 - INFO - Simulated data for 2025-04-15 exported.



Sample of calculated rolling risk metrics:
  ticker  sharpe_ratio  var_param_95                  timestamp
0   AAPL     -0.103734     -0.066550 2025-04-16 09:16:09.951992
1    JNJ     -0.117804     -0.030162 2025-04-16 09:16:09.951992
2   MSFT     -0.060975     -0.039767 2025-04-16 09:16:09.951992
3   NVDA      0.030890     -0.083927 2025-04-16 09:16:09.951992
4   TSLA      0.014199     -0.118032 2025-04-16 09:16:09.951992


2025-04-16 09:16:10,164 - INFO - Simulated data for 2025-04-14 exported.
2025-04-16 09:16:10,177 - INFO - Simulated data for 2025-04-13 exported.
2025-04-16 09:16:10,194 - INFO - Simulated data for 2025-04-12 exported.
2025-04-16 09:16:10,208 - INFO - Simulated data for 2025-04-11 exported.
2025-04-16 09:16:10,228 - INFO - Simulated data for 2025-04-10 exported.
2025-04-16 09:16:10,230 - INFO - PostgreSQL connection closed after simulation.


In [6]:
# EXPORT TO POSTGRESQL DATABASE

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

try:
    rolling_metrics_df.to_sql("rolling_risk_metrics", engine, if_exists='append', index=False)
    logger.info("Rolling risk metrics successfully exported to PostgreSQL.")
except Exception as e:
    logger.error(f"Error exporting rolling risk metrics: {e}")
finally:
    engine.dispose()
    logger.info("PostgreSQL connection closed.")

2025-04-16 09:16:13,369 - INFO - Rolling risk metrics successfully exported to PostgreSQL.
2025-04-16 09:16:13,372 - INFO - PostgreSQL connection closed.
